A proof of concept showing using a genetic algorithm with our environment.
It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

In [ ]:
# Run this cell if you're using colab. Otherwise, skip it.

!git clone https://github.com/platers/meta-transfer-learning.git

import os
os.chdir('meta-transfer-learning')

!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl

!pip install -r requirements.txt
!pip install ran

In [1]:
# some settings that we can tweak:
MAX_STEP_COUNT = 20 # number of steps in each round of DEAP evolution 
POPULATION_SIZE = 3 # population in each round of DEAP evolution 
N_GEN = 3 # number of rounds of DEAP evolution
N_RL_TRANING = 10 # number of rounds of training for RL agent

In [6]:
import array
import random

import numpy as np
from typing import Dict

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym

from environments import SimpleEnv
import importlib
importlib.reload(SimpleEnv)
from environments.SimpleEnv import SimpleEnv #, TODO: add more environments

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks
import ray

import time
start = time.process_time()


In [7]:
ray.shutdown()
ray.init()

n_agents = 3
n_var = 2
training_envs = [
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 0],
    'max_step_count': MAX_STEP_COUNT,
}),
(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [0, 1],
    'max_step_count': MAX_STEP_COUNT,
})]
test_env = (SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'true_reward_weights': [1, 1],
    'max_step_count': MAX_STEP_COUNT,
})

creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
creator.create('Individual', array.array, typecode='d',
               fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register('attr', random.uniform, -1, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr, n_agents * n_var)
toolbox.register('population', tools.initRepeat, list,
                 toolbox.individual)


2020-05-05 16:38:26,784	INFO resource_spec.py:212 -- Starting Ray with 15.09 GiB memory available for workers and up to 7.55 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-05 16:38:27,134	INFO services.py:1170 -- View the Ray dashboard at localhost:8265
/usr/local/google/home/wichersn/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/google/home/wichersn/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [9]:
def evaluate_individual_env(individual, environment, env_config):
    """Runs the environment. All agents have the same policy.
  It returns the total true reward as the fitness.
  """
    #Select random individuals from pop and create the reward weights
    pop = np.array([individual for i in range(n_agents)])
    reward_weights = pop
    env_config['reward_weights'] = reward_weights
    #env is only to get action space and observation space
    env = environment(config=env_config)
    class MyCallbacks(DefaultCallbacks):
        #Callback functions to keep track of true reward while training
        def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
            episode.user_data["true_rewards"] = np.zeros(n_agents)

        def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
            env = base_env
            true_reward = env.env_states[0].env.last_true_reward
            episode.user_data["true_rewards"] += true_reward

        def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
            true_reward = episode.user_data["true_rewards"]
            for i, r in enumerate(true_reward):
                episode.custom_metrics["true_reward_agent_" + str(i)] = r
    
    # settings for the RL agent trainer     
    config={
        "multiagent": {
            "policies": {
            },
            "policy_mapping_fn":  #all agents share a policy
                lambda agent_id:
                    'agent'
        },
        "model": {"fcnet_hiddens": []},
        'env_config': env_config,
        "callbacks": MyCallbacks,
    }
    config['multiagent']['policies']['agent'] = (None, env.observation_space, env.action_space, {})
    trainer = ppo.PPOTrainer(env=environment, config=config)
    
    true_reward_mean = 0
    for i in range(10):
        #print('TRAINING', i)
        true_reward_mean = 0

        #Train the RL agent
        custom_metrics = trainer.train()['custom_metrics']  # distributed training step
        #print(custom_metrics)
        for i in range(n_agents):
            true_reward_mean += custom_metrics['true_reward_agent_' + str(i) + '_mean']
    true_reward_mean /= n_agents
    #print('true reward', trainer.collect_metrics()['custom_metrics']['true_reward_mean'])
    print('Evaluated', individual, 'Fitness', true_reward_mean)
    return true_reward_mean


def evaluate_individual(individual):
    """Runs all environments. 
  returns the average true reward over all environments as the fitness.
  """
    fitness = 0
    for env, config in training_envs:
        fitness += evaluate_individual_env(individual, env, config)
    return (fitness, )


# some setup for Deap
toolbox.register('evaluate', evaluate_individual)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05) # add more noise
toolbox.register('select', tools.selTournament, tournsize=3)

In [ ]:
# pop is a list individual reward function's weights 
pop = toolbox.population(n=POPULATION_SIZE)
# the 10 best individual reward function (could even befround the first round)
hof = tools.HallOfFame(10)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=N_GEN, 
                                   stats=stats, halloffame=hof, verbose=True)

print ('pop', pop)

2020-05-05 16:38:48,495	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:38:48,496	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Evaluated Individual('d', [0.16875954878079402, 0.42780961539696993, -0.5347327810796767, 0.8800975148481187, -0.26218531912788645, 0.3038751945126066]) Fitness -3.7216450901679123


2020-05-05 16:40:18,743	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:40:18,744	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Evaluated Individual('d', [0.16875954878079402, 0.42780961539696993, -0.5347327810796767, 0.8800975148481187, -0.26218531912788645, 0.3038751945126066]) Fitness 0.1605864505935915


2020-05-05 16:41:45,066	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:41:45,068	WARNING util.py:37 -- Install gputil for GPU system monitoring.


...

2020-05-05 16:44:36,384	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:44:36,385	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Evaluated Individual('d', [-0.4808074352793028, 0.02259148574168357, -0.8732926886428483, 0.6063052640326458, -0.42432399341678395, 0.689452033544641]) Fitness -7.0518033943076555


2020-05-05 16:46:01,484	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:46:01,485	WARNING util.py:37 -- Install gputil for GPU system monitoring.


Evaluated Individual('d', [-0.4808074352793028, 0.02259148574168357, -0.8732926886428483, 0.6063052640326458, -0.42432399341678395, 0.689452033544641]) Fitness 2.8099642367963917
gen	nevals	avg     	std    	min     	max     
0  	3     	-2.54158	1.94312	-4.24184	0.178154


2020-05-05 16:47:28,931	INFO trainable.py:217 -- Getting current IP.
2020-05-05 16:47:28,932	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [ ]:
best_individual = hof[0]
print(best_individual)

test_reward = evaluate_individual_env(best_individual, test_env[0], test_env[1])
print(test_reward)

In [ ]:
print(evaluate_individual([0, 0, 1, 0, 1, 0])) #Ideal reward, altruistic agent

In [ ]:
hof[:]

In [ ]:
print(evaluate_individual([1, 0, -1, 0, -1, 0])) #Worst reward, selfish agent

In [ ]:
print("Time Spent = ", (time.process_time() - start)/60, " minutes")